# 图像处理与简易地图栅格化
使用OpenCV对输入图像进行预处理、颜色分割、边界检测，并最终绘制简易地图并栅格化。

In [ ]:
import numpy as np
import cv2
from PIL import Image

## 1. 加载图片并调整大小

In [ ]:
n = 5  # 可变编号
img_path = ""  # 指定图像路径
img = cv2.imread(img_path)
fixed_size = (450, 300)
if img is None:
    print("Can't open image")
    exit()
img = cv2.resize(img, fixed_size)

## 2. CLAHE 对比度增强（在 LAB 空间中处理亮度）

In [ ]:
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
l_enhanced = clahe.apply(l)
lab_enhanced = cv2.merge((l_enhanced, a, b))
img2 = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)

## 3. 转换为BGR空间（暖冷色调整）

In [ ]:
img2 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

## 4. HSV颜色空间转换与掩码设定

In [ ]:
hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
lower_red1 = np.array([0, 100, 45])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 100, 50])
upper_red2 = np.array([180, 255, 255])
lower_orange = np.array([10, 50, 10])
upper_orange = np.array([25, 255, 255])
lower_yellow = np.array([20, 100, 110])
upper_yellow = np.array([40, 255, 255])
lower_brown = np.array([10, 50, 50])
upper_brown = np.array([30, 255, 200])
lower_white = (200, 200, 200)
upper_white = (255, 255, 255)
island_color = (144, 238, 144)
ocean_color = (255, 200, 150)
color_gray = (128, 128, 128)

## 5. 掩码合并与岛屿区域提取

In [ ]:
mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask_orange = cv2.inRange(hsv, lower_orange, upper_orange)
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
warm_mask = mask_red1 | mask_red2 | mask_orange | mask_yellow | mask_brown
island_mask = cv2.bitwise_not(warm_mask)
kernel = np.ones((5, 5), np.uint8)
island_mask = cv2.morphologyEx(island_mask, cv2.MORPH_CLOSE, kernel)

## 6. 轮廓提取与边界绘制

In [ ]:
contours, _ = cv2.findContours(island_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img_boundary = img2.copy()
cv2.drawContours(img_boundary, contours, -1, (0, 0, 255), 2)

## 7. 简易地图绘制并填充海岛和海洋颜色

In [ ]:
img_map = np.zeros(img2.shape, np.uint8) + 255
cv2.drawContours(img_map, contours, -1, (0, 0, 255), 2)
cv2.drawContours(img_map, contours, -1, island_color, thickness=cv2.FILLED)
mask = cv2.inRange(img_map, lower_white, upper_white)
img_map[mask == 255] = ocean_color

## 8. 地图栅格化处理

In [ ]:
img_map2 = img_map.copy()
height, width = img_map2.shape[:2]
grid_size = 20
for x in range(0, width, grid_size):
    cv2.line(img_map2, (x, 0), (x, height), color_gray, 1)
for y in range(0, height, grid_size):
    cv2.line(img_map2, (0, y), (width, y), color_gray, 1)

## 9. 可视化结果并可选择保存

In [ ]:
res = np.hstack((img, img_boundary, img_map))
cv2.imshow("res", res)
cv2.imshow("img", img_map)
cv2.imshow("map", img_map2)
key = cv2.waitKey(0)
if key == ord('s'):
    save_path = fr"E:/sophomore/study/pytorch/test/testhhh_image.jpg"
    cv2.imwrite(save_path, res)
    print(f"图片已保存到: {save_path}")
elif key == ord('q'):
    cv2.destroyAllWindows()
cv2.destroyAllWindows()